# Build Initial Solution Dataset

## Setup

In [ ]:
# Setup and Imports
import json
import pandas as pd

## Load Datasets

In [ ]:
prm800k_train = pd.read_csv("dataset/prm800k-train.csv")
prm800k_train

In [ ]:
prm800k_test = pd.read_csv("dataset/prm800k-test.csv")
prm800k_test

## Get Dataset Samples

*The Strategy*
Idea: Disquality Not Important Datas
Collect Dataset Sample with Slicing from Big to Small

1. Get Rows That Contains -1, 0, 1 (other Rows need dropped)

In [ ]:
import json
import ast

def string_to_dict(input_string):
    input_string = input_string.strip()

    try:
        json_string = input_string.replace("'", '"')
        json_string = json_string.replace('True', 'true').replace('False', 'false').replace('None', 'null')
        return json.loads(json_string)
    except (json.JSONDecodeError, ValueError):
        pass
    
    try:
        return ast.literal_eval(input_string)
    except (ValueError, SyntaxError):
        pass
    
    try:
        if input_string.strip().startswith('{') and input_string.strip().endswith('}'):
            return eval(input_string)
    except:
        pass

    raise ValueError(f"Cannot parse string as dictionary: {input_string[:100]}...")

### 1. Get Rows That Contains -1, 0, 1 (other Rows need dropped)

In [ ]:
def check_rating_completion(step):
    is_neg_1 = False
    is_zero = False
    is_pos_1 = False
    neg_1 = 0
    zero = 0
    pos_1 = 0
    
    for idx, completion in enumerate(step['completions']):
        rating = completion['rating']
        if rating == -1 and not is_neg_1:
            is_neg_1 = True
            neg_1 += 1
        elif rating == 0 and not is_zero:
            is_zero = True
            zero += 1
        elif rating == 1 and not is_pos_1:
            is_pos_1 = True
            pos_1 += 1
    
    return is_neg_1, is_zero, is_pos_1, neg_1, zero, pos_1

In [ ]:
def get_row_contains_all_rating(prm800k_train):
    results = []
    
    for index, row in prm800k_train.iterrows():
        row_neg_1 = 0
        row_zero = 0
        row_pos_1 = 0
        total_steps_have_combination = 0

        try:
            data = string_to_dict(row['label'])
            
            # Process each step
            for step_no, step in enumerate(data['steps']):
                is_neg_1, is_zero, is_pos_1, neg_1, zero, pos_1 = check_rating_completion(step)
                row_neg_1 += neg_1
                row_zero += zero
                row_pos_1 += pos_1
                
                # In one steps combined from 3 category
                if is_neg_1 and is_zero and is_pos_1: 
                    total_steps_have_combination += 1

            if row_neg_1 > 1 and row_zero > 1 and row_pos_1 > 1 and total_steps_have_combination > 0:
                new_result = {
                    **row.to_dict(),
                    "row_neg_1": row_neg_1,
                    "row_zero": row_zero,
                    "row_pos_1": row_pos_1,
                    "total_steps_have_combination": total_steps_have_combination,
                }
                results.append(new_result)
        
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue
    
    # Create DataFrame from results
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
df_contains_all_rating = get_row_contains_all_rating(prm800k_train)
df_contains_all_rating.to_csv('dataset/prm800k-experiment-01-contains-all-rating.csv', index=False)

In [ ]:
df_contains_all_rating

### Versioning:
1. v1.0: row_neg_1 > 0 and row_zero > 0 and row_pos_1 > 0: 37,543 rows
2. v2.0: row_neg_1 > 0 and row_zero > 0 and row_pos_1 > 0 and total_steps_have_combination > 0: 16,535 rows
3. v3.0: row_neg_1 > 1 and row_zero > 1 and row_pos_1 > 1 and total_steps_have_combination > 0: 385 rows